## Applying Minhashing and LSH on papers retrieved

In [1]:
import time
import re
from sklearn.neighbors import LSHForest
import numpy as np
import pandas as pd
from datasketch import MinHash, MinHashLSHForest

## Function to preprocess the papers text

In [2]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

## Function  to get the forest generated on papers data

In [3]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
                
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

## Function to predict the papers based on a text input

In [4]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

## Reading the data from papers.csv and generating the forest(building the model)

In [5]:
db = pd.read_csv('output/papers.csv')
db['text'] = db['title'] + ' ' + db['abstract']

permutations = 128
num_recommendations = 1

forest = get_forest(db, permutations)

It took 16.3326518535614 seconds to build forest.


## Predicting the papers having similarity with the given title

In [6]:
num_recommendations = 5
title = 'Using a neural net to instantiate a deformable model'
result = predict(title, db, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)


It took 0.0069959163665771484 seconds to query forest.

 Top Recommendation(s) is(are) 
 226     Neural Network Weight Matrix Synthesis Using O...
488     A Self-Organizing Integrated Segmentation and ...
203     Analytic Solutions to the Formation of Feature...
976     Using a neural net to instantiate a deformable...
3165    Inferring Neural Firing Rates from Spike Train...
Name: title, dtype: object
